In [22]:
import hashlib
import time

In [7]:
st = b"Blockchain Technology"

In [14]:
hashlib.sha224(st).hexdigest()

'cb4025a30ab13d675d4b0b08122c355a5b76d22f46a5fe1d34dc99aa'

In [16]:
sha_256 = hashlib.sha256(st).hexdigest()
sha_512 = hashlib.sha512(st).hexdigest()
sha3_512 = hashlib.sha3_512(st).hexdigest()
sha3_256 =hashlib.sha3_256(st).hexdigest()

In [18]:
print(sha_512)

b21b9e93f14f980f00dd3d4e45000c7ad6ee41be450f968e36319fd527053eef294ea9311d8d4d6647d1f17a1cef338826b5cccb93f43098657c6569f9b2c54b


In [ ]:
def hash(n,msg):
    start_time = time.time()
    storage = []
    sha_256 = hashlib.sha256(msg).hexdigest()
    while(sha_256[:n] not in storage):
        rando = randomString().encode('utf-8')
        rand_hash = hashlib.sha256(rando).hexdigest()
        storage.append(rand_hash[:n])
#         print(storage)
    storage.append(rand_hash)
    elapsed_time = time.time() - start_time
    return sha_256[:n],elapsed_time
    
print(hash(3,st))

In [40]:
import random
import string

def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

In [77]:
def preimage(n,hashing):
    start_time = time.time()
    rando = randomString().encode('utf-8')
    rand_hash = hashlib.sha256(rando).digest()
    print(rand_hash[:n*4:4])
    print(hashing)
    while(rand_hash[:n*4:4] != hashing):
        rando = randomString().encode('utf-8')
        rand_hash = hashlib.sha256(rando).digest()
#         print(rando)
#         print(rand_hash)
        
    elapsed_time = time.time() - start_time
    return rando.decode(),elapsed_time

print(preimage(2,b"\x00"*3))

b'9O'
b'\x00\x00\x00'


KeyboardInterrupt: 

In [61]:
sha_256 = hashlib.sha256(st).digest()
print(sha_256[:4:4])

b'\xad'


In [4]:
from ecdsa import SigningKey
sk = SigningKey.generate() # uses NIST192p
vk = sk.verifying_key
signature = sk.sign(b"Blockchain Technology")
assert vk.verify(signature, b"Blockchain Technology")

In [328]:
import json
import base64


class Transaction(object):
    
    def __init__(self,sender,receiver,amount,comment,signature):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount
        self.comment = comment
        self.signature = signature
        
    def __eq__(self,transaction):
        if self.sender == transaction.sender and self.receiver==transaction.receiver and self.amount==transaction.amount:
            return True
        return False
        
    @classmethod
    def new(self,sender,receiver,amount,comment,signature):
        t1 = Transaction(sender,receiver,amount,comment,signature)
        return t1
        
    
    def serialize(self):
        props = {}
        props["receiver"] = self.receiver
        props["sender"] =  base64.encodebytes(self.sender.to_string()).decode('ascii')
        props["amount"] = self.amount
        props["comment"] = self.comment
        props["signature"] = self.signature
        return json.dumps(props)
    
    @classmethod
    def deserialize(self,js_string):
        obj = json.loads(js_string)
        obj['sender'] = obj['sender'].encode('ascii')
#         obj['signature'] = obj['signature'].encode('ascii')
        result = Transaction(VerifyingKey.from_string(base64.decodebytes(obj['sender']),curve=NIST384p),obj['receiver'],obj['amount'],obj['comment'],obj['signature'])
        return result 
        
    def sign(self,priv_key):
        s = self.serialize().encode()
        print(s)
        sig = priv_key.sign(s)
        print('finished signing')
        return sig
    
    def validate(self,sig):
        assert self.sender.verify(sig, self.serialize().encode()),"validation failed"
        print('finished validating')
        return
        
        
    

        



In [336]:
# Trans = Transaction('send','rec','amt','comment','sign')
from ecdsa import SigningKey, VerifyingKey , NIST384p
sk = SigningKey.generate(NIST384p) # uses NIST192p
vk = sk.verifying_key
Trans = Transaction(vk,'rec','amt','comment','sign')
json1 = Trans.serialize()
json1_des = Trans.deserialize(json1)
print(Trans==json1_des)

True


In [337]:
sig = Trans.sign(sk)
# fake_trans = Transaction(vk,'asd','asd','asdt','sign')
# fake_sig = fake_trans.sign(sk)
print(Trans.validate(sig))

b'{"receiver": "rec", "sender": "F9A0d4N0LNgViGTzUMmqSLW8yeVgekZxAPp5luNyfyMcQfUyMRhRqunFiS7x1Y3CGNcio+e5VOSd\\nhbNL0BBRrzfkdbOR8TLTT6pTR35PnV+osmUUlCYkDfZ1b8dgopKp\\n", "amount": "amt", "comment": "comment", "signature": "sign"}'
finished signing
finished validating
None


In [234]:
import os
import hashlib

class MarkleTree:
    def __init__(self, root):
        self._linelength = 30
        self._root = root
        self._mt = {}
        self._hashlist = {}
        self._tophash = ''
        self.__MT__()

    def Line(self):
        print(self._linelength*'-')

    def PrintHashList(self):
        self.Line()
        for item, itemhash in self._hashlist.items():
            print("%s %s" % (itemhash, item))
        self.Line()
        return

    def PrintMT(self, hash):
        value = self._mt[hash]
        item = value[0]
        child = value[1]
        print("%s %s" % (hash, item))
        if not child:
            return
        for itemhash, item in child.items():  
            print("    -> %s %s" % (itemhash, item))
        for itemhash, item in child.items():  
            self.PrintMT(itemhash)

    def MT(self):
        for node, hash in self._hashlist.items():
            items = self.GetItems(node)
            value = []
            value.append(node)
            list = {}
            for item in items:
                if node == self._root:
                    list[self._hashlist[item]] = item
                else: 
                    list[self._hashlist[os.path.join(node, item)]] = os.path.join(node, item)
            value.append(list)
            self._mt[hash] = value
        self._tophash = self._hashlist[self._root]

    def __MT__(self):
        self.HashList(self._root)
        #self.PrintHashList()
        self.MT()
        print("Merkle Tree for %s: " % self._root)
        self.PrintMT(self._tophash)
        self.Line()

    def md5sum(self, data):
        m = hashlib.md5()
        fn = os.path.join(self._root, data)
        if os.path.isfile(fn):
            try:   
                f = file(fn, 'rb')
            except:
                return 'ERROR: unable to open %s' % fn
            while True:
                d = f.read(8096)
                if not d:
                    break
                m.update(d)
            f.close()
        else:
            m.update(data)
        return m.hexdigest()

    def GetItems(self, directory):
        value = []
        if directory != self._root:
            directory = os.path.join(self._root, directory)
        if os.path.isdir(directory):
            items = os.listdir(directory)
            for item in items:
                value.append(item)
                #value.append(os.path.join(".", item))
            value.sort()
        return value
    
    def HashList(self, rootdir):
        self.HashListChild(rootdir)
        items = self.GetItems(rootdir)
        if not items:
            self._hashlist[rootdir] = ''
            return
        s = ''
        for subitem in items:
            s = s + self._hashlist[subitem]
        self._hashlist[rootdir] = self.md5sum(s)

    def HashListChild(self, rootdir):
        items = self.GetItems(rootdir)
        if not items:
            self._hashlist[rootdir] = ''
            return
        for item in items:
            itemname = os.path.join(rootdir, item)
            if os.path.isdir(itemname):
                self.HashListChild(item)
                subitems = self.GetItems(item)
                s = ''
                for subitem in subitems:
                    s = s + self._hashlist[os.path.join(item, subitem)]
                if rootdir == self._root:
                    self._hashlist[item] = self.md5sum(s)
                else:
                    self._hashlist[itemname] = self.md5sum(s)
            else:
                if rootdir == self._root:
                    self._hashlist[item] = self.md5sum(item)
                else:
                    self._hashlist[itemname] = self.md5sum(itemname)
 

def MTDiff(mt_a, a_tophash, mt_b, b_tophash):
    if a_tophash == b_tophash:
        print("Top hash is equal for %s and %s" % (mt_a._root, mt_b._root))
    else:
        a_value = mt_a._mt[a_tophash] 
        a_child = a_value[1]    # retrive the child list for merkle tree a
        b_value = mt_b._mt[b_tophash] 
        b_child = b_value[1]    # retrive the child list for merkle tree b

        for itemhash, item in a_child.items():
            try:
                if b_child[itemhash] == item:
                    print("Info: SAME : %s" % item)
            except:
                print("Info: DIFFERENT : %s" % item)
                temp_value = mt_a._mt[itemhash]
                if len(temp_value[1]) > 0:      # check if this is a directory
                    diffhash = list(set(b_child.keys()) - set(a_child.keys()))
                    MTDiff(mt_a, itemhash, mt_b, diffhash[0])
                    
                    
if __name__ == "__main__":
    mt_a = MarkleTree('testA')
    print(mt_a._mt)
    mt_b = MarkleTree('testB')
    MTDiff(mt_a, mt_a._tophash, mt_b, mt_b._tophash)

Merkle Tree for testA: 
 testA
------------------------------
{'': ['testA', {}]}
Merkle Tree for testB: 
 testB
------------------------------
Top hash is equal for testA and testB


In [8]:
class Block(Transaction) :
    
    def __init__(self,header,hash_tree):
        self.header = header
        self.hash_tree = hash_tree
        return
    

        

In [ ]:
class Blockchain(Transaction):
    
    def __init__(self,blocks,target):
        self.blocks = blocks
        self.target = target
        
    def add():
        
        

In [ ]:
target = '\x00\x00\x0f\xff\xff\xff...\xff'